Contains code for DP fair epsnet for Rectangles. 

In [1]:
import sys
sys.path.append('../../..')

import numpy as np
import random
import pandas as pd
import time
from matplotlib import pyplot as plt

from core.points import *
from core.ranges import *
from core.fairness import FairnessMeasure
from algorithms.fairness.fair_epsnet import *
from core.ranges import get_range_space
from core.verification import is_fair_epsnet

Sampling method:

In [15]:
def report_fair_epsnet_sample(n, m, eps, ratios, c1):
    success_prob = 0.9
    
    
    color_counts = []
    for ratio in ratios:
        color_counts.append(int(n * ratio))
    
    points = []
    for i, count in enumerate(color_counts):
        points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=i) for _ in range(count)]
    
    ranges = []
    for _ in range(m):
        xmin = random.uniform(0, 1)
        ymin = random.uniform(0, 1)
        xmax = random.uniform(xmin, 1)  # Ensure xmax > xmin
        ymax = random.uniform(ymin, 1)  # Ensure ymax > ymin
        ranges.append(RectangleRange(xmin=xmin, ymin=ymin, xmax=xmax, ymax=ymax))
    rangespace = get_range_space(points, ranges)
    vc = 2

    heavy_ranges = []
    for r in rangespace:
        if len(r) >= n * eps:
            heavy_ranges.append(r)
            
    print(f"n: {n}, m: {m}, vc: {vc}, epsilon: {eps}, number of heavy ranges: {len(heavy_ranges)}")

    # for _ in range(10):
    start = time.time()
    epsnet = build_fair_epsnet_sample(
        points=points,
        rangespace=rangespace,
        vc=vc,
        epsilon=eps,
        color_ratios=ratios,
        fairconfig=FairConfig(k=len(ratios), fairness=FairnessMeasure.DP),
        success_prob=success_prob,
        weights=None,
        c1=c1
    )
    end = time.time()

    print(f"Number of points in epsnet: {len(epsnet)}, time taken: {end - start:.6f} seconds")

    success = is_fair_epsnet(epsnet=epsnet, rangespace=rangespace, epsilon=eps, points=points)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, 
            success_prob, 
            eps, 
            end - start, 
            success, 
            len(heavy_ranges), 
            len([p for p in epsnet if p.color == 0]), len([p for p in epsnet if p.color == 1]),
            len(ratios), # k
            ratios,
            [len([p for p in epsnet if p.color == i]) for i in range(len(ratios))], # color counts
            len(epsnet)
            )

In [ ]:
# report_fair_epsnet_sample(2**12, 2**12, 0.6, [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625], c1=1/2)  # Example usage

In [25]:
n_values = [2**12]#, 2**11, 2**12]
m_values = [2**12]
# eps_values = [0.2, 0.3, 0.4, 0.5, 0.6]
eps_values = [0.4]
rates = [
    [0.5, 0.5],
    # [0.6, 0.4],
    # [0.7, 0.3],
    # [0.8, 0.2],
    [0.25, 0.25, 0.25, 0.25],
    [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125],
    [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625],
    [0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 
     0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 
     0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 
     0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 
     0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 
     0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 0.0625 / 2, 
     0.0625 / 2, 0.0625 / 2]
]

aggregated_results = {
    "n": [],
    "m": [],
    "prob": [],
    "eps": [],
    "time": [],
    "success": [],
    "heavy_ranges": [],
    "tries": [],
    "red_points": [],
    "blue_points": [],
    "k": [],
    "ratios": [],
    "counts": [],
    "size": []
}

for n in n_values:
    for m in m_values:
        for eps in eps_values:
            for ratio in rates:
                tries = 20
                print(f"Running for n={n}, m={m}, k={len(ratio)}")
                result = report_fair_epsnet_sample(n, m, eps, ratio, c1=1)
                # while not result[5] and tries > 0:
                    # print("Retrying...")
                    # result = report_fair_epsnet_sample(n, m, eps, ratio, c1=1)
                    # tries -= 1
                
                aggregated_results["n"].append(result[0])
                aggregated_results["m"].append(result[1])
                aggregated_results["prob"].append(result[2])
                aggregated_results["eps"].append(result[3])
                aggregated_results["time"].append(result[4])
                aggregated_results["success"].append(result[5])
                aggregated_results["heavy_ranges"].append(result[6])
                aggregated_results["tries"].append(10 - tries)
                aggregated_results["red_points"].append(result[7])
                aggregated_results["blue_points"].append(result[8])
                aggregated_results["k"].append(result[9])
                aggregated_results["ratios"].append(result[10])
                aggregated_results["counts"].append(result[11])
                
                aggregated_results["size"].append(result[12])

result = pd.DataFrame(aggregated_results)

Running for n=4096, m=4096, k=2
n: 4096, m: 4096, vc: 2, epsilon: 0.4, number of heavy ranges: 59
[build_fair_epsnet_sample] epsnet size m: 213, v: 3
[_augment_epsnet] epsnet colors count:
	[_augment_epsnet] Color 0: 124
	[_augment_epsnet] Color 1: 89
[_augment_epsnet] Color 0 to add: 195
[_augment_epsnet] Color 1 to add: 230
Number of points in epsnet: 638, time taken: 0.000318 seconds
Not a fair eps-net!
Success: False
Running for n=4096, m=4096, k=4
n: 4096, m: 4096, vc: 2, epsilon: 0.4, number of heavy ranges: 65
[build_fair_epsnet_sample] epsnet size m: 213, v: 3
[_augment_epsnet] epsnet colors count:
	[_augment_epsnet] Color 0: 57
	[_augment_epsnet] Color 1: 58
	[_augment_epsnet] Color 2: 49
	[_augment_epsnet] Color 3: 49
[_augment_epsnet] Color 0 to add: 102
[_augment_epsnet] Color 1 to add: 101
[_augment_epsnet] Color 2 to add: 110
[_augment_epsnet] Color 3 to add: 110
Number of points in epsnet: 636, time taken: 0.000375 seconds
Not a fair eps-net!
Success: False
Running for n

In [ ]:
# result.to_csv("dp_fair_epsnet_sampling_size_vs_k.csv", index=False)

In [ ]:
# result.to_csv("dp_fair_epsnet_rectangle_sample.csv", index=False)

Sketch and merge:

In [23]:
def report_fair_epsnet_sketch_merge(n, m, eps, ratios):
    success_prob = 0.9
    
    
    color_counts = []
    for ratio in ratios:
        color_counts.append(int(n * ratio))
    
    points = []
    for i, count in enumerate(color_counts):
        points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=i) for _ in range(count)]
    
    ranges = []
    for _ in range(m):
        xmin = random.uniform(0, 1)
        ymin = random.uniform(0, 1)
        xmax = random.uniform(xmin, 1)  # Ensure xmax > xmin
        ymax = random.uniform(ymin, 1)  # Ensure ymax > ymin
        ranges.append(RectangleRange(xmin=xmin, ymin=ymin, xmax=xmax, ymax=ymax))
    rangespace = get_range_space(points, ranges)
    vc = 2

    heavy_ranges = []
    for r in rangespace:
        if len(r) >= n * eps:
            heavy_ranges.append(r)
            
    print(f"n: {n}, m: {m}, vc: {vc}, epsilon: {eps}, number of heavy ranges: {len(heavy_ranges)}")

    # for _ in range(10):
    start = time.time()
    epsnet = build_fair_epsnet_sketch_merge(
        points=points,
        rangespace=rangespace,
        vc=vc,
        epsilon=eps,
        fairconfig=FairConfig(k=len(ratios), fairness=FairnessMeasure.DP),
        c1=0
    )
    end = time.time()

    print(f"Number of points in epsnet: {len(epsnet)}, time taken: {end - start:.6f} seconds")

    success = is_fair_epsnet(epsnet=epsnet, rangespace=rangespace, epsilon=eps, points=points)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, 
            success_prob, 
            eps, 
            end - start, 
            success, 
            len(heavy_ranges), 
            len([p for p in epsnet if p.color == 0]), len([p for p in epsnet if p.color == 1]),
            len(ratios), # k
            ratios
            )

In [ ]:
report_fair_epsnet_sketch_merge(2**10, 2**10, 0.3, [0.5, 0.5])  # Example usage

In [31]:
n_values = [2**10]
m_values = [2**9, 2**10, 2**11, 2**12]
eps_values = [0.2, 0.3, 0.4, 0.5, 0.6]
rates = [
    [0.5, 0.5]
]

aggregated_results = {
    "n": [],
    "m": [],
    "prob": [],
    "eps": [],
    "time": [],
    "success": [],
    "heavy_ranges": [],
    "tries": [],
    "red_points": [],
    "blue_points": [],
    "k": [],
    "ratios": []
}

for n in n_values:
    for m in m_values:
        for eps in eps_values:
            for ratio in rates:
                tries = 20
                print(f"Running for n={n}, m={m}, eps={eps}, ratio={ratio}")
                result = report_fair_epsnet_sketch_merge(n, m, eps, ratio)
                while not result[5] and tries > 0:
                    print("Retrying...")
                    result = report_fair_epsnet_sketch_merge(n, m, eps, ratio)
                    tries -= 1
                
                aggregated_results["n"].append(result[0])
                aggregated_results["m"].append(result[1])
                aggregated_results["prob"].append(result[2])
                aggregated_results["eps"].append(result[3])
                aggregated_results["time"].append(result[4])
                aggregated_results["success"].append(result[5])
                aggregated_results["heavy_ranges"].append(result[6])
                aggregated_results["tries"].append(10 - tries)
                aggregated_results["red_points"].append(result[7])
                aggregated_results["blue_points"].append(result[8])
                aggregated_results["k"].append(result[9])
                aggregated_results["ratios"].append(result[10])

result = pd.DataFrame(aggregated_results)

Running for n=1024, m=512, eps=0.2, ratio=[0.5, 0.5]
n: 1024, m: 512, vc: 2, epsilon: 0.2, number of heavy ranges: 50
[build_fair_epsnet_sketch_merge] epsnet size m: 506
[build_fair_epsnet_sketch_merge] partition size p: 512
[build_fair_epsnet_sketch_merge] Starting sketch-and-merge...
[_sketch_merge] pair: 1 / 1 of total nodes: 2
[_greedy_discrepancy_halving] counter: 512 / 512

Number of points in epsnet: 512, time taken: 3.011530 seconds
Success: True
Running for n=1024, m=512, eps=0.3, ratio=[0.5, 0.5]
n: 1024, m: 512, vc: 2, epsilon: 0.3, number of heavy ranges: 24
[build_fair_epsnet_sketch_merge] epsnet size m: 306
[build_fair_epsnet_sketch_merge] partition size p: 512
[build_fair_epsnet_sketch_merge] Starting sketch-and-merge...
[_sketch_merge] pair: 1 / 1 of total nodes: 2
[_greedy_discrepancy_halving] counter: 512 / 512

Number of points in epsnet: 512, time taken: 2.987373 seconds
Success: True
Running for n=1024, m=512, eps=0.4, ratio=[0.5, 0.5]
n: 1024, m: 512, vc: 2, epsil

In [ ]:
# result.to_csv("dp_fair_epsnet_rectangle_sketch_merge.csv", index=False)

Naive:

In [44]:
def report_fair_epsnet_naive(n, m, eps, ratios):
    success_prob = 0.9
    
    
    color_counts = []
    for ratio in ratios:
        color_counts.append(int(n * ratio))
    
    points = []
    for i, count in enumerate(color_counts):
        points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=i) for _ in range(count)]
    
    ranges = []
    for _ in range(m):
        xmin = random.uniform(0, 1)
        ymin = random.uniform(0, 1)
        xmax = random.uniform(xmin, 1)  # Ensure xmax > xmin
        ymax = random.uniform(ymin, 1)  # Ensure ymax > ymin
        ranges.append(RectangleRange(xmin=xmin, ymin=ymin, xmax=xmax, ymax=ymax))
    rangespace = get_range_space(points, ranges)
    vc = 2

    heavy_ranges = []
    for r in rangespace:
        if len(r) >= n * eps:
            heavy_ranges.append(r)
            
    print(f"n: {n}, m: {m}, vc: {vc}, epsilon: {eps}, number of heavy ranges: {len(heavy_ranges)}")

    # for _ in range(10):
    start = time.time()
    epsnet = build_fair_epsnet_naive(
        points=points,
        rangespace=rangespace,
        vc=vc,
        epsilon=eps,
        fairconfig=FairConfig(k=len(ratios), fairness=FairnessMeasure.DP),
        success_prob=success_prob,
        weights=None,
        # c1=c1
    )
    end = time.time()

    print(f"Number of points in epsnet: {len(epsnet)}, time taken: {end - start:.6f} seconds")

    success = is_fair_epsnet(epsnet=epsnet, rangespace=rangespace, epsilon=eps, points=points)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, 
            success_prob, 
            eps, 
            end - start, 
            success, 
            len(heavy_ranges), 
            len([p for p in epsnet if p.color == 0]), len([p for p in epsnet if p.color == 1]),
            len(ratios), # k
            ratios
            )

In [45]:
n_values = [2**9, 2**10, 2**11, 2**12]
m_values = [2**9, 2**10, 2**11, 2**12]
eps_values = [0.2, 0.3, 0.4, 0.5, 0.6]
rates = [
    [0.5, 0.5],
    [0.6, 0.4],
    [0.7, 0.3],
    [0.8, 0.2],
    [0.25, 0.25, 0.25, 0.25],
    [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125],
    [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625]
]

aggregated_results = {
    "n": [],
    "m": [],
    "prob": [],
    "eps": [],
    "time": [],
    "success": [],
    "heavy_ranges": [],
    "tries": [],
    "red_points": [],
    "blue_points": [],
    "k": [],
    "ratios": []
}

for n in n_values:
    for m in m_values:
        for eps in eps_values:
            for ratio in rates:
                tries = 20
                print(f"Running for n={n}, m={m}, eps={eps}, ratio={ratio}")
                result = report_fair_epsnet_naive(n, m, eps, ratio)
                while not result[5] and tries > 0:
                    print("Retrying...")
                    result = report_fair_epsnet_naive(n, m, eps, ratio)
                    tries -= 1
                
                aggregated_results["n"].append(result[0])
                aggregated_results["m"].append(result[1])
                aggregated_results["prob"].append(result[2])
                aggregated_results["eps"].append(result[3])
                aggregated_results["time"].append(result[4])
                aggregated_results["success"].append(result[5])
                aggregated_results["heavy_ranges"].append(result[6])
                aggregated_results["tries"].append(10 - tries)
                aggregated_results["red_points"].append(result[7])
                aggregated_results["blue_points"].append(result[8])
                aggregated_results["k"].append(result[9])
                aggregated_results["ratios"].append(result[10])

result = pd.DataFrame(aggregated_results)

Running for n=512, m=512, eps=0.2, ratio=[0.5, 0.5]
n: 512, m: 512, vc: 2, epsilon: 0.2, number of heavy ranges: 37
[build_fair_epsnet_naive] epsnet size m: 506, v: 2
[_augment_epsnet] epsnet colors count:
	[_augment_epsnet] Color 0: 246
	[_augment_epsnet] Color 1: 260
[_augment_epsnet] Color 0 to add: 260
[_augment_epsnet] Color 1 to add: 246
Number of points in epsnet: 702, time taken: 0.001398 seconds
Success: True
Running for n=512, m=512, eps=0.2, ratio=[0.6, 0.4]
n: 512, m: 512, vc: 2, epsilon: 0.2, number of heavy ranges: 33
[build_fair_epsnet_naive] epsnet size m: 506, v: 2
[_augment_epsnet] epsnet colors count:
	[_augment_epsnet] Color 0: 316
	[_augment_epsnet] Color 1: 190
[_augment_epsnet] Color 0 to add: 291
[_augment_epsnet] Color 1 to add: 214
Number of points in epsnet: 699, time taken: 0.001346 seconds
Not a fair eps-net!
Success: False
Retrying...
n: 512, m: 512, vc: 2, epsilon: 0.2, number of heavy ranges: 46
[build_fair_epsnet_naive] epsnet size m: 506, v: 2
[_augmen

In [ ]:
# result.to_csv("dp_fair_epsnet_rectangle_naive.csv", index=False)